In [1]:
import time
import numpy as np
from tqdm.notebook import tqdm
import logging
logging.basicConfig(level=logging.INFO, force=True)  # `force=True` перезаписывает настройки
   
# teleop
from echo.teleoperators import Echo, EchoConfig, TestTeleoperator

# robots
from echo.robots.robot import make_robot_from_config, BimanualRobotConfig
from echo.robots.ur3 import UR3Config
from echo.robots.test_robot import TestRobotConfig

# cameras
from echo.cameras.realsense import RealSenseCameraConfig
from echo.cameras.opencv import OpenCVCameraConfig
from echo.cameras.test_camera import TestCameraConfig

# dataset
from echo.data_utils.lerobot_dataset import LeRobotDataset
from echo.data_utils.dataset_record_config import DatasetRecordConfig
from echo.data_utils.utils import hw_to_dataset_features
from echo.data_utils.data_recorder import DataRecorder
from echo.data_utils.video_utils import VideoEncodingManager

from echo.utils import sanity_check_dataset_robot_compatibility

INFO:numexpr.utils:Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
INFO:numexpr.utils:NumExpr defaulting to 16 threads.


In [2]:
# CONFIGURATION
RECORD_DATA = True
RESUME = False

left_base_pose = np.array(
    [
        0.0002879959065467119,
        -1.5719226042376917,
        -1.5715907255755823,
        3.1422481536865234,
        -1.5707929770099085,
        512.0812350745701,
    ]
)

right_base_pose = np.array(
    [
        -0.001450840626851857,
        -1.5725587050067347,
        1.5713553428649902,
        0.0008134841918945312,
        1.5712484121322632,
        9.42502719560732,
    ]
)

camera_config = {"main_view": RealSenseCameraConfig()}

robot_config = TestRobotConfig(arm = "left", base_pose=left_base_pose, cameras=camera_config)

echo_config = EchoConfig()
dataset_config = DatasetRecordConfig(root = "test_dataset")

In [ ]:

list(b.keys())

['a', 'b']

In [7]:
b = {"a": float, "b": float}
a = dict(zip(list(b.keys()), [1, 2]))
a

{'a': 1, 'b': 2}

In [ ]:
echo = TestTeleoperator(echo_config)

robot = make_robot_from_config(robot_config)
robot.connect()

action_features = hw_to_dataset_features(robot.action_features, "action", dataset_config.use_videos)
obs_features = hw_to_dataset_features(robot.observation_features, "observation", dataset_config.use_videos)
dataset_features = {**action_features, **obs_features}

if RECORD_DATA:
    if RESUME:
        #  Resume from existing dataset
        dataset = LeRobotDataset(
            root=dataset_config.root,
            batch_encoding_size=dataset_config.video_encoding_batch_size,
        )
        sanity_check_dataset_robot_compatibility(dataset, robot, dataset_config.fps, dataset_features)
        
    else:
        # Create empty dataset
        dataset = LeRobotDataset(
            fps= dataset_config.fps,
            root=dataset_config.root,
            robot_type=robot.name,
            features=dataset_features,
            use_videos=dataset_config.use_videos,
        )

    data_recorder = DataRecorder(dataset = dataset, robot = robot)

In [ ]:
# Main loop for teleoperation and dataset collection.
# The loop continuously reads data from the echo and processes it.

# def main_loop(
#     echo, robot, dataset, target_HZ=20
# ):
    
TARGET_PERIOD = 1.0 / (dataset_config.fps + 0.095) # Target period 
data_timeout = 5.0 # Timeout for waiting for data from the echo device.
# The progress bar is initially hidden and will be revealed after a few iterations.
pbar = None
counter = 0

task = "some new task"

In [ ]:
# with VideoEncodingManager(dataset):
while True:

    # Record the start time for the current iteration.
    # This is used to calculate the elapsed time for maintaining the target frequency.
    start_time = time.perf_counter()

    data_from_echo = echo.get_data()
    if data_from_echo is None:
        continue

    action = robot.calculate_action(data_from_echo)

    if dataset is not None:
        data_recorder.record(data_from_echo, action, task)

    # send action to the robot
    robot.send_action(action)
    
    # If the hidden count reaches 3, enable the progress bar.
    if counter > 3 and pbar is None:
        pbar = tqdm(desc="Collecting Dataset", unit="it")
    else:
        counter+=1

    # If the progress bar is initialized, update it.
    if pbar is not None:
        pbar.update(1)

    # Calculate the sleep time to maintain the target frequency.
    elapsed = time.perf_counter() - start_time
    sleep_time = TARGET_PERIOD - elapsed
    # print(f"Elapsed time: {elapsed:.3f}s") # Debugging line
    # print("First iteration") if i < 3 else print(f"Elapsed time: {elapsed:.3f}s") # Debugging line
    if sleep_time > 0:
        time.sleep(sleep_time)

### Test video writer

In [ ]:
import av
from pathlib import Path
from PIL import Image
import logging
from typing import Optional, Union
import numpy as np

class RealTimeVideoWriter:
    def __init__(
        self,
        output_path: Union[Path, str],
        fps: int = 30,
        vcodec: str = "h264",
        pix_fmt: str = "yuv420p",
        crf: int = 23,
        g: Optional[int] = None,
        frame_size: Optional[tuple[int, int]] = None,
    ):
        """
        Инициализация видео-райтера для записи в реальном времени.
        
        :param output_path: Путь для сохранения видео
        :param fps: Частота кадров
        :param vcodec: Видеокодек (h264, hevc, libsvtav1)
        :param pix_fmt: Формат пикселей
        :param crf: Constant Rate Factor (качество)
        :param g: GOP size
        :param frame_size: Размер кадра (width, height)
        :param enable_logging: Включить логирование
        """
        # Отключение логов av
        av.logging.set_level(av.logging.ERROR)
        
        # Проверка кодеков
        if vcodec not in ["h264", "hevc", "libsvtav1"]:
            raise ValueError(f"Unsupported video codec: {vcodec}")
        
        if vcodec in ["libsvtav1", "hevc"] and pix_fmt == "yuv444p":
            logging.warning(f"Incompatible pixel format 'yuv444p' for {vcodec}, using 'yuv420p'")
            pix_fmt = "yuv420p"
        
        self.output_path = Path(output_path)
        self.output_path.parent.mkdir(parents=True, exist_ok=True)
        
        self.fps = fps
        self.vcodec = vcodec
        self.pix_fmt = pix_fmt
        self.crf = crf
        self.g = g
        self.frame_size = frame_size
        
        self._output = None
        self._stream = None
        self._frame_count = 0
        self._is_open = False
    
    def __enter__(self):
        """Контекстный менеджер для автоматического закрытия"""
        self.open()
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        """Автоматическое закрытие при выходе из контекста"""
        self.close()
    
    def open(self) -> None:
        """Открытие видеофайла для записи"""
        if self._is_open:
            return
        
        self._output = av.open(str(self.output_path), mode="w")
        
        # Настройка параметров кодека
        video_options = {"crf": str(self.crf)}
        if self.g is not None:
            video_options["g"] = str(self.g)
        
        self._stream = self._output.add_stream(self.vcodec, rate=self.fps, options=video_options)
        self._stream.pix_fmt = self.pix_fmt
        
        self._is_open = True
        self._frame_count = 0
        
        logging.info(f"Video writer opened: {self.output_path}")
    
    def add_frame(self, frame: Union[Image.Image, np.ndarray]) -> None:
        """
        Добавление кадра в видео.
        
        :param frame: PIL Image или numpy array (HWC, BGR/RGB)
        """
        if not self._is_open:
            raise RuntimeError("Video writer is not open. Call open() first.")
        
        # Конвертация numpy array в PIL Image если нужно
        if isinstance(frame, np.ndarray):
            if frame.ndim == 3 and frame.shape[2] == 3:
                frame = Image.fromarray(frame)
            else:
                raise ValueError("Unsupported numpy array format")
        
        # Установка размера кадра при первом добавлении
        if self.frame_size is None:
            self.frame_size = frame.size
            self._stream.width = self.frame_size[0]
            self._stream.height = self.frame_size[1]
        else:
            # Проверка совпадения размеров
            if frame.size != self.frame_size:
                raise ValueError(f"Frame size {frame.size} doesn't match expected size {self.frame_size}")
        
        # Создание и кодирование кадра
        av_frame = av.VideoFrame.from_image(frame)
        packet = self._stream.encode(av_frame)
        
        if packet:
            self._output.mux(packet)
        
        self._frame_count += 1
    
    def close(self) -> None:
        """Завершение записи и сохранение видео"""
        if not self._is_open:
            return
        
        try:
            # Флашим энкодер
            packet = self._stream.encode()
            if packet:
                self._output.mux(packet)
            
            # Закрываем выходной файл
            self._output.close()
                
        except Exception as e:
            logging.error(f"Error closing video writer: {e}")
            # Удаляем временный файл в случае ошибки
            if self.output_path.exists():
                self.output_path.unlink()
            raise
        finally:
            self._is_open = False
    
    def save(self) -> None:
        """Алиас для close()"""
        self.close()
    
    @property
    def frame_count(self) -> int:
        """Количество добавленных кадров"""
        return self._frame_count
    
    @property
    def is_open(self) -> bool:
        """Статус записи"""
        return self._is_open
    
    def abort(self) -> None:
        """Прерывание записи (удаляет временный файл)"""
        if self._is_open:
            self._output.close()
            if self.output_path.exists():
                self.output_path.unlink()
            self._is_open = False
            logging.info("Video recording aborted")

In [ ]:
from echo.cameras.realsense import RealSenseCameraConfig, RealSenseCamera

In [ ]:
config = RealSenseCameraConfig(camera_fps=30, frame_height=480, frame_width=640)

In [ ]:
cam = RealSenseCamera(config)

In [ ]:
cam.connect()

In [ ]:
cam.get_frame().shape

In [ ]:
# Создание и использование
with RealTimeVideoWriter("output.mp4", fps=30, crf=23) as writer:
    while(True):
        # Генерация или получение кадра
        frame = cam.get_frame()
        writer.add_frame(frame)

In [ ]:
writer.frame_size